In [62]:
from dotenv import load_dotenv
env = load_dotenv()

In [8]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from IPython.display import Image
import IPython

from typing import TypedDict, Annotated, Dict, Any
import operator
import json

In [7]:
class AgentState(TypedDict):
    restaurant: str
    output: str

In [70]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults()

In [94]:
# Download PDF tool
from langchain.tools import tool
from langchain.pydantic_v1 import BaseModel, Field
from urllib.request import urlopen

class SearchInput(BaseModel):
    download_url: str = Field(description="the URL of the menu pdf")
    name: str = Field(description="the name of the restaurants")

@tool("download-pdf", args_schema=SearchInput, return_direct=True)
def download_pdf(download_url: str, filename: str):
    """Download a pdf file from a given url and filename"""
    response = urlopen(download_url)
    file = open("pdf/"+filename+".pdf", 'wb')
    file.write(response.read())
    file.close()

AttributeError: 'str' object has no attribute 'parent_run_id'

In [27]:
tools = [search_tool]

In [71]:
response = tool.invoke("CRISP rochester menu")

In [72]:
response

[{'url': 'https://crisprochester.com/menus/',
  'content': 'CRISP Rochester menus. Lunch, dinner, cocktails, wine, beer, and brunch menus. Skip to content. 819 S. Clinton Ave. Rochester, NY. 14620 +1 (585) 978-7237. Menus. ... Lunch / Dinner Menu; Brunch Menu; Dessert Menu; Drink Menus. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna ...'},
 {'url': 'https://crisprochester.com/',
  'content': 'Lunch / Dinner Menu. THURS - SAT 11:30am - 10:00pm. SUN / MON 4:00pm - 9:00pm. Brunch. 11:00am - 2:45pm. Crisp Rochester is an American food restaurant serving modern comfort food, with a southern influence, craft cocktails & local beers served in a cozy and chic setting in the South Wedge of Rochester.'},
 {'url': 'https://crisprochester.com/wp-content/uploads/Lunch-and-Dinner.pdf',
  'content': 'Crisp iceberg wedge, tomato, green onion, bacon, house buttermilk bleu, topped with crispy onions and blue cheese crumbles. Fried Chic

In [63]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone()

In [64]:
pc.create_index(
    name="quickstart",
    dimension=5120, # Lamma 2 embedding dim
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [ ]:
class Model:
    # Define the model
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_node("output-parser", self.parse_output_to_string)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: "output-parser"}
        )
        graph.add_edge("action", "llm")
        graph.add_edge("output-parser", END)
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    # Check if llm requires action 
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    # Run a tool ordered by the model
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}
    # Perform inference on the gathered context
    def classify(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    # def parse_output_to_string(self, state: AgentState):
    #     message = state['messages'][-1]
    #     json_output = JsonOutputParser(pydantic_object=Restaurants).invoke(message)
    #     return {'json_output': json_output}

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
"""You are an AI tasked with classifying compatible diet types from a menu.

----------
{ menu }
----------
You output MUST be structured list of one or more of the provided diet types,
{ diet_types }
----------
DO NOT respond as an AI assistant, only return a list of compatible diet types. 
You may return no diet types if no fit, or one diet type if only one fits.

EXAMPLE:
["diet_type_1", "diet_type_2", ...]
"""
)

prompt.format(context=context)

In [ ]:
llm = OllamaFunctions(
    model="phi3", 
    keep_alive=-1,
    format="json"
    )
)

model = Model(llm, tools, system=prompt)

In [ ]:
Image(model.graph.get_graph().draw_png())

In [ ]:
input = [HumanMessage(content="CRISP Rochester")]
result = model.graph.invoke({"restaurant": input})